## Building Fast Queries on a CSV

In this project i own an online laptop store and want to build a way to answer a few different business questions about my inventory.

The goal of this project is to create a class that represents my inventory. The methods in that class will implement the queries that i want to answer about my inventory. I will also preprocess that data to make those queries run faster.

Here are some queries that i will want to answer:

- Given a laptop id, find the corresponding data.
- Given an amount of money, find whether there are two laptops whose total price is that given amount.
- Identify all laptops whose price falls within a given budget.

1. Imports:

In [1]:
import csv
import time
import random

In [2]:
with open ("laptops.csv", "r", encoding="UTF-8") as file:
    data = list(csv.reader(file))
    header = data[0]
    rows = data[1:]

2. EDA:

In [3]:
print(header)

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']


In [4]:
print(rows[:5])

[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD', 'Intel Iris Plus Graphics 650', 'macOS', '1.37kg', '1803']]


3. Creating an Inventory Class:

In [5]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])

In [6]:
inventory = Inventory("laptops.csv")

In [7]:
print(inventory.header)

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']


In [8]:
print(len(inventory.rows))

1303


4. Finding a Laptop From the Id:

In [9]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
            else:
                None

In [10]:
inventory = Inventory("laptops.csv")

In [11]:
print(inventory.get_laptop_from_id("3362737"))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]


In [12]:
print(inventory.get_laptop_from_id("3362736"))

None


5. Improving Id Lookups:

In [13]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
            else:
                return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return None

In [14]:
inventory = Inventory("laptops.csv")

In [15]:
print(inventory.get_laptop_from_id_fast("3362737"))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]


In [16]:
print(inventory.get_laptop_from_id_fast("3362736"))

None


6. Comparing the Performance:

In [17]:
ids = [str(random.randint(1000000, 9999999) for i in range(10000))]

In [18]:
inventory = Inventory("laptops.csv")

In [19]:
total_time_no_dict = 0

for id_number in ids:
    start = time.time()
    inventory.get_laptop_from_id(id_number)
    end = time.time()
    total_time_no_dict += (end-start)
    
total_time_dict = 0
    
for id_number in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(id_number)
    end = time.time()
    total_time_dict += (end-start)

In [20]:
print(total_time_no_dict)

2.1457672119140625e-06


In [21]:
print(total_time_dict)

1.1920928955078125e-06


It can be easily spotted that using a dictionary is faster option for finding a laptop.

7. Two Laptop Promotion:

In [22]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
            else:
                return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for i in self.rows:
            for j in self.rows:
                if i[-1] + j[-1] == dollars:
                    return True
        return False

In [23]:
inventory = Inventory("laptops.csv")

In [24]:
print(inventory.check_promotion_dollars(1000))

True


In [25]:
print(inventory.check_promotion_dollars(442))

False


8. Optimizing Laptop Promotion:

In [26]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
            else:
                return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for i in self.rows:
            for j in self.rows:
                if i[-1] + j[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False

In [27]:
inventory = Inventory("laptops.csv")

In [28]:
print(inventory.check_promotion_dollars_fast(1000))

True


In [29]:
print(inventory.check_promotion_dollars_fast(442))

False


9. Comparing Promotion Functions:

In [30]:
prices = [random.randint(100, 5000) for i in range (100)]

In [31]:
inventory = Inventory("laptops.csv")

In [32]:
total_time_no_set = 0

for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += (end-start)
    
total_time_set = 0

for price in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += (end-start)

In [33]:
print(total_time_no_set)

0.8823287487030029


In [34]:
print(total_time_set)

0.0004515647888183594


It can be easily spotted that using set is way faster.

10. Finding Laptops Within a Budget:

In [35]:
def row_price(row):
    return row[-1]

class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key = row_price)
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
            else:
                return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for i in self.rows:
            for j in self.rows:
                if i[-1] + j[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, price): # Step 2
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                   
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            current_price = self.rows_by_price[range_middle][-1]
            if current_price > price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= price:                  
            return -1                                   
        return range_start

In [36]:
inventory = Inventory("laptops.csv")

In [37]:
print(inventory.find_first_laptop_more_expensive(1000))

683


In [38]:
print(inventory.find_first_laptop_more_expensive(10000))

-1
